# Packages Import

In [ ]:
## loading required packages

import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from utils.labels import map_class_to_rgb, adjust_mask
import segmentation_models as sm
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from utils.util import *
from arch.deeplabv3plus import Deeplabv3



# Paths and Constants

In [ ]:
##### paths


## update data path

base_data_dir = 'dataset'


saved_models = 'docs_models'

train_path = os.path.join(base_data_dir, 'data')

classes_path = os.path.join(base_data_dir, 'classes.csv')


## parmeters

## change based on your resources
batch_sz =4
num_classes = 20
DIM=300
seed = 1
epoch_num=200

# Read Labels file

In [ ]:
classes = pd.read_csv(classes_path,index_col='name')

# data generators

In [ ]:
# Data generator
train_generator_fn, val_generator_fn, train_length, validation_length = train_valid_generator(train_path, seed, batch_sz, DIM)

# Model (change here)

In [ ]:
## best model so far for training , you may change weights and backbone according to available options inside "arch/deeplabv3plus.py"



##  you may change used model if you want by importing any model from arch folder and using it here instead

model = Deeplabv3(weights='cityscapes',
    input_shape=(DIM, DIM, 3),
    classes=num_classes,
    backbone='xception',
    activation='softmax')


In [ ]:
# model.summary()

# freeze layers (optional)

In [ ]:
## xception exit_flow_block2_separable_conv1_depthwise
## mobilenetv2 expanded_conv_16_project

# freeze_model_layers(model,'exit_flow_block2_separable_conv1_depthwise')

# model compilation and fitting (you may change here if needed)

In [ ]:
model_name='deeplabv3_fulltraining'  ## output model file name 

cp_name = os.path.join(saved_models,
                            model_name+'_epoch_{epoch:03d}_valLoss_{val_loss:.4f}.h5')

model_checkpoint = ModelCheckpoint(filepath=cp_name
                                   , monitor='val_loss',verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(
    monitor = "val_loss", 
    patience = 20,
    verbose = 1, 
    factor = 0.2, 
    min_lr = 0.000001,
)


In [ ]:
# necassary import for our model optimizer which is based on tensfolow 1.x
from tensorflow.python.keras.optimizers import adam_v2


model.compile(
    optimizer= adam_v2.Adam(learning_rate=1e-4),
    loss=sm.losses.cce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
    
)

history = model.fit(
    train_generator_fn(),
    steps_per_epoch=train_length // batch_sz
  , validation_steps=validation_length // batch_sz,
   epochs=1,
   validation_data= val_generator_fn(),  
   callbacks=[model_checkpoint,reduce_lr]
)




# plotting loss and MeanIou

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
iou_score = history.history['iou_score']
val_iou_score = history.history['val_iou_score']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, iou_score, 'r', label='Training iou_score')
plt.plot(epochs, val_iou_score, 'b', label='Validation iou_score')
plt.title('Training and validation iou_score')
plt.legend()

plt.show()

# loading Created Model

In [ ]:
from keras.models import load_model

## change with new saved model name
model = load_model(filepath='docs_models/deeplabv3_fulltraining_epoch_173_valLoss_0.4395.h5', custom_objects={
    "categorical_crossentropy_plus_jaccard_loss":sm.losses.cce_jaccard_loss
    ,"iou_score":sm.metrics.iou_score})

# predicting Validation data

In [ ]:
plot_predictions (train_path,model,DIM,mode='valid')